# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Подключим все необходимое

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
nltk.download('averaged_perceptron_tagger')
from lightgbm import LGBMClassifier

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/tmp/ipykernel_213/2556472392.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

### Загрузим и ознакомимся с данными

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Странный столбец повторяющий индексы, но в общем он нам не мешает.

In [5]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

**Мини-вывод:**
* Непонятный столбец индексов
* Есть дисбаланс классов, пока исправлять это не будем, посмотрим на результаты моделей

### Подготовим выборки к работе

Напишем функции разделения, и обработки текста.


In [6]:
def target_features_split(data):
    target = data['toxic']
    features = data.drop(['toxic'], axis=1)
    return target, features

In [7]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [8]:
m = WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = " ".join([m.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())


In [9]:
data['lemm_text'] = data['text'].apply(lemmatize_text)

data = data.drop(['text'], axis=1)
data = data.drop(['Unnamed: 0'], axis=1)
del m

In [10]:
data.head()

,toxic,lemm_text
0,0,explanation why the edits make under my userna...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not try to edit war it s ju...
3,0,more i ca n t make any real suggestion on impr...
4,0,you sir be my hero any chance you remember wha...


<font color='purple'><b>Комментарий студента</b> странно, но без POS тегов и токенизации результаты были получше, по типу 0.77 и 0.78</font>

In [11]:
random_state = 12345

In [12]:
train, test = train_test_split(data, test_size=0.2, random_state=random_state, stratify=data['toxic'])

In [13]:
target_train, features_train = target_features_split(train)
target_test, features_test =  target_features_split(test)

In [14]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
vec = TfidfVectorizer(stop_words=stopwords)
features_train = vec.fit_transform(features_train['lemm_text'])
features_test = vec.transform(features_test['lemm_text'])

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
print(features_train.shape)
print(features_test.shape)

(127433, 137036)
(31859, 137036)


## Обучение

### Обучим и сравним результаты моделей

In [16]:
results = pd.DataFrame(columns=['model', 'f1'])

In [17]:
lr_model = LogisticRegression()
parameters = [{'C': [8.7, 8.8, 8.6]}]

lr_model_gscv = GridSearchCV(lr_model, param_grid=parameters, scoring='f1', n_jobs=-1, 
                   cv=5, verbose=True)
lr_model_gscv.fit(features_train, target_train)
print(lr_model_gscv.best_params_)
results = results.append({'model': 'LogisticRegression', 'f1': lr_model_gscv.best_score_}, ignore_index=True)


Fitting 5 folds for each of 3 candidates, totalling 15 fits
{'C': 8.6}


In [18]:
lsvc_model= LinearSVC(random_state=42, class_weight='balanced')

parameters = [{'C': [0.5, 0.4, 0.3]}]

lsvc_model_gscv = GridSearchCV(lsvc_model, param_grid=parameters, scoring='f1', n_jobs=-1, 
                   cv=5, verbose=True)
lsvc_model_gscv.fit(features_train, target_train)
print(lsvc_model_gscv.best_params_)
results = results.append({'model': 'LinearSVC', 'f1': lsvc_model_gscv.best_score_}, ignore_index=True)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
{'C': 0.5}


In [19]:
results

,model,f1
0,LogisticRegression,0.762080
1,LinearSVC,0.754739


Результаты очень похожи, протестируем обе

In [20]:
predict = lr_model_gscv.best_estimator_.predict(features_test)
print('F1:', f1_score(target_test, predict))
predict = lsvc_model_gscv.best_estimator_.predict(features_test)
print('F1:', f1_score(target_test, predict))

F1: 0.7767857142857143
F1: 0.7613318410744263


Все таже схожесть результатов, потенциал конечно больше у LinearSVC, и обучилась она быстрее.

## Выводы

**Общий вывод по моделям:**
Модели показывают практически одинаковый результат удовлетворяющий заказчика, по этому можно предложить в работу любую из них, конечно у LinearSVC(при дополнительной необходимости) можно еще поиграть с гиперпараметрами.

**Проделанная работа:**
* Загрузили и ознакомились с данными, обнаружили странный столбец который в последствии был удален, так же имелся дисбаланс классов, но модели справились
* Подготовили текст для работы моделей (лемматизация,приведение к нужному формату, очистка от лишних символов и стоп слов, векторизация, добавили POS теги) 
* Разделили на выборки
* Обучили с подбором параметров 2 модели, которые показали практически одинаковый результат
* Провели тестирование моделей где так же был получен схожий результат